In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import torch.optim as optim


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


dataset_path = '/content/drive/MyDrive/project/miniImageNet'
train_set = torchvision.datasets.ImageFolder(root=os.path.join(dataset_path, 'train'), transform=transform)
val_set = torchvision.datasets.ImageFolder(root=os.path.join(dataset_path, 'val'), transform=transform)
test_set = torchvision.datasets.ImageFolder(root=os.path.join(dataset_path, 'test'), transform=transform)


train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_set.classes))
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)


            optimizer.zero_grad()


            outputs = model(inputs)
            loss = criterion(outputs, labels)


            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')


train_model(model, criterion, optimizer, num_epochs=5)


project_folder = '/content/drive/MyDrive/project'
model_save_path = os.path.join(project_folder, 'resnet18_miniImageNet.pth')
torch.save(model.state_dict(), model_save_path)


print(f"Model saved to: {model_save_path}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 172MB/s]


Epoch 1/5, Loss: 1.3128431661219537
Epoch 2/5, Loss: 0.8590132582791244
Epoch 3/5, Loss: 0.610519960220856
Epoch 4/5, Loss: 0.4343314125002185
Epoch 5/5, Loss: 0.32685659160908265
Model saved to: /content/drive/MyDrive/project/resnet18_miniImageNet.pth


In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class SimpleDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.img_labels = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        self.img_labels.sort()
        self.folder_path = folder_path
        self.transform = transform
        self.labels = list(set(file_name.split('_')[0] for file_name in self.img_labels))
        self.label_to_index = {label: index for index, label in enumerate(self.labels)}

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.folder_path, self.img_labels[idx])
        image = Image.open(img_path).convert("RGB")
        label = self.img_labels[idx].split('_')[0]
        label_index = self.label_to_index[label]
        if self.transform:
            image = self.transform(image)
        return image, label_index


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_dataset = SimpleDataset('/content/drive/My Drive/project/EuroSAT/trainset', transform=transform)
test_dataset = SimpleDataset('/content/drive/My Drive/project/EuroSAT/testset', transform=transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features


pretrained_dict = torch.load('/content/drive/My Drive/project/resnet18_miniImageNet.pth', map_location=device)


pretrained_dict = {k: v for k, v in pretrained_dict.items() if 'fc' not in k}


model.load_state_dict(pretrained_dict, strict=False)


model.fc = nn.Linear(num_ftrs, len(train_dataset.labels))


model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def fine_tune_model(model, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')


fine_tune_model(model, criterion, optimizer, num_epochs=5)


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total}%')


model_save_path = '/content/drive/My Drive/project/resnet18_EuroSAT.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Fine-tuned model saved to: {model_save_path}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/5, Loss: 1.8507752418518066
Epoch 2/5, Loss: 0.757851779460907
Epoch 3/5, Loss: 0.34761515259742737
Epoch 4/5, Loss: 0.14797437191009521
Epoch 5/5, Loss: 0.061268970370292664
Accuracy of the model on the test images: 86.66666666666667%
Fine-tuned model saved to: /content/drive/My Drive/project/resnet18_EuroSAT.pth
